In [1]:
import csv
import pandas as pd
import numpy as np


Extracting dataset for training

In [120]:
dataset=pd.read_csv('reviews_100.csv')

In [121]:
print(dataset['sentiment'].value_counts())

negative    50
positive    50
Name: sentiment, dtype: int64


Stopwords removal and tokenization


In [122]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/somyadahiaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/somyadahiaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [231]:

# dataset['joined_review'] = dataset['cleaned_review'].apply(lambda words: ' '.join(words))

# vectorizer = CountVectorizer(max_features=500, stop_words='english')
# X = vectorizer.fit_transform(dataset['joined_review'])  # Features
# y = dataset['sentiment']  # Labels






stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text =re.sub(r'<[^>]+>', '', text)
    remove_punctuation=[char for char in text if char not in string.punctuation]
    remove_punctuation=''.join(remove_punctuation)
    text= [word.lower() for word in remove_punctuation.split() if word.lower() not in stop_words]
    return text

# Test
print(remove_stopwords("This is an example <br> text<\br>."))

['example', 'text']


In [159]:


dataset.iloc[:,0].apply(remove_stopwords)


0     [viewed, first, two, nights, coming, imdb, loo...
1     [said, several, times, least, watching, eric, ...
2     [without, doubt, worse, comicbook, movie, ever...
3     [firstly, would, like, point, people, criticis...
4     [compact, crime, drama, good, amount, action, ...
                            ...                        
95    [wasnt, saw, sidney, pollack, picture, ever, c...
96    [daniel, auteuil, queen, margot, much, better,...
97    [got, movie, ebay, mainly, im, gay, love, til,...
98    [preposterous, sequel, stretches, credibility,...
99    [bugs, bunny, accidentally, ends, south, pole,...
Name: review, Length: 100, dtype: object

In [223]:
from sklearn.feature_extraction.text import CountVectorizer

transformer=CountVectorizer(analyzer=remove_stopwords,max_features=500).fit(dataset['review'])

vectorised=transformer.transform(dataset['review'])

In [224]:
print(vectorised)

  (0, 0)	1
  (0, 8)	1
  (0, 34)	2
  (0, 44)	1
  (0, 46)	2
  (0, 65)	1
  (0, 75)	1
  (0, 129)	1
  (0, 148)	1
  (0, 154)	2
  (0, 158)	1
  (0, 168)	1
  (0, 177)	1
  (0, 178)	1
  (0, 180)	1
  (0, 187)	1
  (0, 231)	1
  (0, 239)	1
  (0, 249)	2
  (0, 252)	1
  (0, 255)	1
  (0, 277)	1
  (0, 295)	1
  (0, 300)	1
  (0, 305)	2
  :	:
  (99, 2)	1
  (99, 47)	1
  (99, 51)	1
  (99, 56)	3
  (99, 59)	1
  (99, 119)	1
  (99, 121)	1
  (99, 124)	2
  (99, 126)	1
  (99, 179)	1
  (99, 181)	1
  (99, 199)	1
  (99, 207)	1
  (99, 219)	1
  (99, 246)	1
  (99, 255)	1
  (99, 287)	1
  (99, 295)	1
  (99, 339)	1
  (99, 342)	1
  (99, 369)	1
  (99, 374)	1
  (99, 384)	1
  (99, 440)	2
  (99, 444)	1


In [225]:
X=vectorised.toarray()

print(X)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]]


In [226]:
# X=vectorised
dataset['label'] = dataset['sentiment'].map({'positive': 1, 'negative': 0})
y=dataset['label']
print(X.shape)


(100, 500)


In [227]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.2, random_state=42)

In [228]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.6500


In [229]:
def predict_review_sentiment(model, vectorizer, review):
    review_vector = vectorizer.transform([review])
    prediction = model.predict(review_vector)[0]
    return prediction

In [230]:
review_texts = ["An absolutely stunning movie with a powerful message!","Absolutely amazing work, good movie.", "Worst movie ever, not recommended", "good movie , thriller and intersting to watch with friends." ]

for review in review_texts:
    predicted=predict_review_sentiment(model, transformer, review)
    print(review)

    if predicted==1:
        print("Predicted Sentiment: positive")

    else:
        print("Predicted Sentiment: negative")

       





An absolutely stunning movie with a powerful message!
Predicted Sentiment: negative
Absolutely amazing work, good movie.
Predicted Sentiment: positive
Worst movie ever, not recommended
Predicted Sentiment: negative
good movie , thriller and intersting to watch with friends.
Predicted Sentiment: positive


Here when i am using max_features =500 the test accuracy is 0.65 whereas when there is no limit on max features, the accuracy becomes 0.55.
